In [1]:
import sys
import os
import pandas as pd
ruta = r"C:\Users\mariajose_chinchilla\Documents\GitHub\Reservas\utils"
sys.path.append(os.path.abspath(ruta))
from utils import resultado_percentiles, resultado_por_montos
import seaborn as sn
import matplotlib.pyplot as plt
import scipy.stats as stats
import math

In [2]:
score = pd.read_excel("../db/actualizado/mspas.xlsx")

In [3]:
mascara_deudores = score["default_30"] == 1

In [4]:
score[score["default_30"] == 1]["rciInternoFinalDeudor"].mean()

45.57671875

In [ ]:
distribuciones = [
    "binorm", #binormal
    'norm',          # Distribución Normal
    't',             # Distribución t de Student
    'expon',         # Distribución Exponencial
    'uniform',       # Distribución Uniforme
    'lognorm',       # Distribución Log-Normal
    'weibull_min',   # Distribución Weibull mínima
    'weibull_max',   # Distribución Weibull máxima
    'beta',          # Distribución Beta
    'gamma',         # Distribución Gamma
    'chi2',          # Distribución Chi-Cuadrado
    'pareto',        # Distribución Pareto
    'rayleigh',      # Distribución Rayleigh
    'gumbel_r',      # Distribución Gumbel derecha
    'gumbel_l',      # Distribución Gumbel izquierda
    'f',             # Distribución F de Fisher
    'binom',         # Distribución Binomial
    'poisson',       # Distribución Poisson
    'nbinom'         # Distribución Binomial Negativa
]



In [ ]:
df = score.loc[mascara_deudores]
for distribucion in distribuciones:
    try:
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        stats.probplot(score["scoreDeudorInstallmentQnt"], dist=distribucion, sparams=(len(df)-1,), plot=ax[0])
        ax[0].set_title(f'Q-Q Plot para Distribución {distribucion}')

        plt.show()
    except:
        pass

In [ ]:
from scipy.stats import norm
parametros = norm.fit(score["scoreDeudorInstallmentQnt"].astype("float"))
sigma_estimado = parametros[1]  # El segundo valor es el parámetro de escala (sigma)

print(f"El valor estimado de sigma es: {sigma_estimado}")


percentil_deseado = 99  # Por ejemplo, queremos calcular el percentil 90
valor_percentil = norm.ppf(percentil_deseado / 100, scale=sigma_estimado)

print(f"El valor del {percentil_deseado} percentil es: {valor_percentil}")

In [ ]:
import numpy as np
from scipy.stats import t
h0 = "x < critico dado que es moroso"
df = score.loc[mascara_deudores]
media = df["scoreDeudorInstallmentQnt"].mean()
desviacion = score["scoreDeudorInstallmentQnt"].std()
alpha = 0.05
z = t.ppf(alpha, len(score) - 1)

z_stat = ((360 - media) / desviacion) * np.sqrt(len(score))
p_valor = t.cdf(z_stat, len(score) - 1)
corte = 360 + z * desviacion / np.sqrt(len(score))

if z_stat > z:
    print("Rechazar hipótesis: hay suficiente evidencia para decir que el promedio de score de morosos es mayor")
if z_stat <= z:
    print("No hay evidencia para rechazar:  no hay evidencia para decir que el promedio de score de morosos es mayor")

print(p_valor)
print(corte)
print(z)

In [ ]:
mascara_nuevos = ["scoreDeudorInstallmentQnt"]
proyeccion = score.loc[mascara_nuevos]


In [ ]:
score.columns

In [ ]:
proporcion_muestral = len(score[score["default_30"] == 1]) / len(score)
significancia = 0.05
z = abs(norm.ppf(1 - significancia))
desviacion_estimador = (proporcion_muestral * (1 - proporcion_muestral)) / len(score)
proporcion_1 = proporcion_muestral - z * desviacion_estimador
proporcion_2 = proporcion_muestral + z * desviacion_estimador
print((proporcion_1, proporcion_2))
print(f"en cantidad es {proporcion_1 * len(score)}")

In [ ]:
score["mora en corte"] = np.where()
desembolso_promedio = score["rciInternoFinalDeudor"].mean()
significancia = 0.05
z = abs(norm.ppf(1 - significancia))
desviacion_estimador = desembolso_promedio / len(df)
media1 = desembolso_promedio - z * desviacion_estimador
media2 = desembolso_promedio + z * desviacion_estimador
print((media1, media2))

In [20]:
mascara_a = (score["default_30"] == 1) & (score["default_60"] == 0)
mascara_b = (score["default_30"] == 1) & (score["default_60"] == 1) & (score["default_90"] == 0)
mascara_c = (score["default_30"] == 1) & (score["default_60"] == 1) & (score["default_90"] == 1)
propa = len(score[mascara_a]) / len(score)
propb = len(score[mascara_b]) / len(score)
propc = len(score[mascara_c]) / len(score)
cata = score[(mascara_a)]["DESEMBOLSO_NETO"].sum() * 0.45 * 0.036
catb = score[(mascara_b)]["DESEMBOLSO_NETO"].sum() * 0.45 * 0.126
catc = score[(mascara_c)]["DESEMBOLSO_NETO"].sum() * 0.45 * 0.295
cata + catb + catc
# score[(mascara_a | mascara_b | mascara_c)]["DESEMBOLSO_NETO"].sum()


405973.71519749996

In [13]:
score[score["scoreDeudorInstallmentQnt"] <= 360]["DESEMBOLSO_NETO"].sum()

9292485.46

In [ ]:
len(score[(score["scoreDeudorInstallmentQnt"] <= 420) & (score["scoreDeudorInstallmentQnt"] > 360)]) 

In [ ]:
score["grupo"] = pd.cut(score["scoreDeudorInstallmentQnt"], bins=40)
conteo = score.groupby(by=["grupo", "default_30"]).size().unstack(fill_value=0).reset_index()
conteo.columns = ["GRUPO", "BUENOS", "MALOS"]
conteo["MALO ACUMULADO"] = conteo["MALOS"].cumsum()
conteo["BUENOS ACUMULADO"] = conteo["BUENOS"].cumsum()
conteo["PROPORCION"] = conteo["MALO ACUMULADO"] / conteo["BUENOS ACUMULADO"]
# conteo.drop("MALOS", inplace=True, axis=1)
# conteo.drop("BUENOS", inplace=True, axis=1)
print(conteo[["GRUPO", "PROPORCION"]])

In [ ]:
actual = score[score["scoreDeudorInstallmentQnt"] <= 420]
print(f"les pido codeudor a {len(actual)}")
print(f"proporcion mora es {len(score[score["default_30"] == 1]) / len(score)}")

In [ ]:
# le pido codeudor a 2.8% de la población y eso representa el 3.3% de mora
# ahora le pediría codeudor al 1% de la población y esperaríamos mora de 
965 / len(score)